<a href="https://colab.research.google.com/github/lucianosilva-github/compiladores/blob/main/COMPILADORES_AULA_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 08**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**

*   Continuar o estudo do analisador semântico
*   Entender o processo de verificação de tipos
*   Aplicar o design pattern Visitor para realizar o processo de verificação de tipos


In [ ]:
!pip install rply

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

A seguir, temos a implementação das classes dos nós da árvore sintática, já com o método accept para receber os visitors:

In [ ]:
from rply.token import BaseBox

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass
    

class Sub(BinaryOp):
  pass
   

class Mul(BinaryOp):
  pass
    

class Div(BinaryOp):
  pass
    


A seguir, temos um visitor que percorre a árvore sintática e resolve a expressão contida nela:

In [ ]:
class Visitor(object):
    pass


class Eval(Visitor):
    def visit_number(self, i):
        return i.value
        
    def visit_add(self, a):
        return a.left.accept(self) + a.right.accept(self)

    def visit_sub(self, a):
        return a.left.accept(self) - a.right.accept(self)

    def visit_mul(self, a):
        return a.left.accept(self) * a.right.accept(self)

    def visit_div(self, a):
        return a.left.accept(self) / a.right.accept(self)


Finalmente, temos o analisador sintático:

In [ ]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])    
    ]
)

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

Para testar o visitor, fazemos a análise sintática, geramos a árvore sintática e, finalmente, passamos o visitor de resolução a ela:

In [ ]:
arvore=parser.parse(lexer.lex('1+2*5'))
print(arvore.accept(Eval()))

11


Implemente e teste um visitor para decorar esta árvore sintática, utilizando o seguinte conjunto de regras de tipo:

NUMBER --> int

int + int --> int

int - int --> int

int * int --> int

int / int --> int


In [ ]:
class Decorator(Visitor):
    def visit_number(self, i):
        i.decor_type="int"
        
          

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
          

    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

In [ ]:
arvore.accept(Decorator())
print(arvore.decor_type)


int


**ATIDADE DA AULA PASSADA**

Na aula passada, aumentamos a gramática vista em aula para incluir declaração de variáveis com os tipos int e string, assim como um comando de atribuição. Foi solicitada a implementação de dois visitor:

(1) um visitor para montar a tabela de símbolos

(2) um visitor para decorar a árvore sintática. Suponha que NUMBER tenha o tipo int.


\<var-decls\> ::= \<var-decl\> 

       | \<var-decl\> \<var-decls\>

\<var-decl\> ::= \<type\> ID ;

\<type\> ::= int | string 

\<atrib\> ::= ID = \<expression\> ;

\<expression\> ::= ID

      | NUMBER

      | \<expression\> "+" \<expression\>
 
      | \<expression\> "-" \<expression\>
 
      | \<expression\> "*" \<expression\>
 
      | \<expression\> "/" \<expression\>
 
      | "(" <expression> ")"

Inicialmente, vamos implementar um analisador léxico para esta gramática aumentada:

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

A seguir, temos a implementação das classes dos nós da árvore sintática, já com o método accept para receber os visitors:

In [ ]:
from rply.token import BaseBox

class VarDecls(BaseBox):

    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visit_vardecls(self)

class VarDecl(BaseBox):

    def __init__(self, tp,id):
        self.tp = tp
        self.id = id

    def accept(self, visitor):
        visit_vardecl(self)

class Atrib(BaseBox):

    def __init__(self, id,expr):
        self.id = expr
        self.expr = expr

    def accept(self, visitor):
        visit_atrib(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass
    

class Sub(BinaryOp):
  pass
   

class Mul(BinaryOp):
  pass
    

class Div(BinaryOp):
  pass
    

In [ ]:
#faça sua implementação aqui